In [76]:
p_storage_dir = "../../storage"

In [77]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from pathlib import Path

DA_STORAGE_DIR = Path(f"{p_storage_dir}/dados_abertos")

# Disciplinas Ministradas

In [78]:
data_types = defaultdict(lambda: str)
data_types.update({"cod_pauta": np.int32, "ano_letivo": np.int32})

disciplinas_ministradas = pd.read_csv(
    DA_STORAGE_DIR / "transformed/disciplinas_ministradas.csv",
    delimiter=";",
    dtype=data_types
)

disciplinas_ministradas.head()

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,sigla_disciplina,nome_e_carga_horaria,periodo_oferta,nome_e_siape_docente,nome_e_siape_docente_auxiliar
0,117006,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21504,DPAA-AL.22,Biologia I (27),Semestral,Fernanda Keley Silva Pereira Navarro (2116924),NaN
1,117007,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21507,DPAA-AL.25,Epidemiologia (27),Semestral,Danielly Bandeira Lopes (2128584),NaN
2,117008,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21501,DPAA-AL.19,História I (27),Semestral,Tiago Gomes de Araújo (2119988),NaN
3,117009,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21505,DPAA-AL.23,Informática Básica I (27),Semestral,Ricardo Marques de Carvalho (2144097),NaN
4,117010,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21500,DPAA-AL.18,Língua Portuguesa I (54),Semestral,Monique Leite Araujo (1622509),NaN


## Atomização da coluna "nome_e_carga_horaria"

Essa coluna pode ser atomizada em duas colunas separadas, uma sendo "nome_disciplina" e "carga_horaria_disciplina".

In [79]:
nome_carga_horaria_disciplinas = disciplinas_ministradas["nome_e_carga_horaria"]

Verificar se existem disciplinas com o nome ausente:

In [80]:
if nome_carga_horaria_disciplinas.isna().unique() == np.ndarray(shape=(1,1), dtype=bool, buffer=np.array([False])):
    print("Não existem disciplinas sem nome e carga horária especificados no dataset.")
else:
    print("Existem disciplinas com o nome e carga horária ausente no dataset.")

Não existem disciplinas sem nome e carga horária especificados no dataset.


Verificar se existem disciplinas com carga horária fora do padrão esperado para a atomização:

In [81]:
has_carga_horaria_fora_padrao = False
disciplinas_fora_padrao = []


for nome_carga_horaria in nome_carga_horaria_disciplinas:
    carga_horaria = nome_carga_horaria.split(" ")[-1]
    
    # Regex que captura cargas horárias inteiras "(54)" e cargas horárias fracionadas (40.5)
    carga_horaria_pattern = r"(^\(\d{1,3}\)$|^\(\d{1,3}\.\d{1}\)$)"
    
    if not re.match(carga_horaria_pattern, carga_horaria.strip()):
        disciplinas_fora_padrao.append(nome_carga_horaria)
        is_carga_horaria_fora_padrao = True
        
if has_carga_horaria_fora_padrao:
    print("As seguintes disciplinas estão com a carga horária fora do padrão:")
    
    for disciplina_fora_padrao in disciplinas_fora_padrao:
            print(f"• {disciplina_fora_padrao}")        
else:
    print("Todas as disciplinas estão em conforme com o padrão esperado pela atomização.")
    

Todas as disciplinas estão em conforme com o padrão esperado pela atomização.


In [82]:
nome_e_carga_horaria_raw = disciplinas_ministradas.values[:, 12]

nome_e_carga_horaria_dict = {
    "nome_disciplina": [],
    "carga_horaria_disciplina": []
}

for nome_e_carga_horaria in nome_e_carga_horaria_raw:
    nome_e_carga_horaria_splitted = nome_e_carga_horaria.split(" ")
    carga_horaria = re.sub(r"[()]", "", nome_e_carga_horaria_splitted[-1].strip())
    nome = " ".join(nome_e_carga_horaria_splitted[:len(nome_e_carga_horaria_splitted) - 1]).strip()
    
    nome_e_carga_horaria_dict["nome_disciplina"].append(nome)
    nome_e_carga_horaria_dict["carga_horaria_disciplina"].append(carga_horaria)

nome_e_carga_horaria_atomized = pd.DataFrame(nome_e_carga_horaria_dict)

nome_e_carga_horaria_atomized.head()

,nome_disciplina,carga_horaria_disciplina
0,Biologia I,27
1,Epidemiologia,27
2,História I,27
3,Informática Básica I,27
4,Língua Portuguesa I,54


In [83]:
disciplinas_ministradas.drop("nome_e_carga_horaria", axis=1, inplace=True)

In [84]:
disciplinas_ministradas.insert(11, "nome_disciplina", nome_e_carga_horaria_atomized["nome_disciplina"])
disciplinas_ministradas.insert(12, "carga_horaria_disciplina", nome_e_carga_horaria_atomized["carga_horaria_disciplina"])

In [85]:
disciplinas_ministradas[(disciplinas_ministradas["campus_ofertante"] != "Câmpus Virtual de Ead") & 
                       (disciplinas_ministradas["ano_letivo"] < 2017)].tail()

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta,nome_e_siape_docente,nome_e_siape_docente_auxiliar
112715,154849,Câmpus Valparaíso,2016,2,20162.CTEP.5P,5,Técnico Integrado ao Ensino Médio em Eletrotéc...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Valparaíso ...,24130,Instalações Elétricas Prediais I,54,DPAA-VP.156,Semestral,Wagner Jose Nascimento de Oliveira (2296458),NaN
112716,154850,Câmpus Valparaíso,2016,2,20162.CTEP.5P,5,Técnico Integrado ao Ensino Médio em Eletrotéc...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Valparaíso ...,24074,Instrumentação Industrial,54,DPAA-VP.120,Semestral,Ariel Caleb Fernandes Souza (2296976),NaN
112717,154851,Câmpus Valparaíso,2016,2,20162.CTEP.5P,5,Técnico Integrado ao Ensino Médio em Eletrotéc...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Valparaíso ...,24112,Língua Portuguesa V,27,DPAA-VP.138,Semestral,Bernadette Marie da Silveira Rodrigues (1721377),NaN
112718,154852,Câmpus Valparaíso,2016,2,20162.CTEP.5P,5,Técnico Integrado ao Ensino Médio em Eletrotéc...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Valparaíso ...,24117,Matemática V,27,DPAA-VP.143,Semestral,Lucimeire Alves de Carvalho (2337849),NaN
112719,154853,Câmpus Valparaíso,2016,2,20162.CTEP.5P,5,Técnico Integrado ao Ensino Médio em Eletrotéc...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Valparaíso ...,24124,Sociologia do Trabalho II,27,DPAA-VP.150,Semestral,Alda Lucia Monteiro de Souza (2262632),NaN


## Tratamento das colunas "nome_e_siape_docente" e "nome_e_siape_docente_auxiliar"

No atual contexto essas colunas possuem dados redundantes, pois no nosso grafo nós já temos os dados dos docentes, bem como suas matrículas SIAPE. Tendo isso em vista nós podemos remover o dado do nome (tanto para o docente efetivo como para o auxiliar) e deixar apenas a matrícula SIAPE, para depois usar esse dado para relacionar a disciplina ministrada com um docente.

In [86]:
dados_docente_efetivo_e_auxiliar = disciplinas_ministradas.loc[:, ["nome_e_siape_docente", "nome_e_siape_docente_auxiliar"]]

dados_docente_efetivo_e_auxiliar.head()

,nome_e_siape_docente,nome_e_siape_docente_auxiliar
0,Fernanda Keley Silva Pereira Navarro (2116924),NaN
1,Danielly Bandeira Lopes (2128584),NaN
2,Tiago Gomes de Araújo (2119988),NaN
3,Ricardo Marques de Carvalho (2144097),NaN
4,Monique Leite Araujo (1622509),NaN


In [87]:
for index, row in dados_docente_efetivo_e_auxiliar.iterrows():
    columns_labels = ["nome_e_siape_docente", "nome_e_siape_docente_auxiliar"]

    for column_label in columns_labels:
        if not pd.isna(row[column_label]):
            filtered_siape = re.sub(r"[()]", "", row[column_label].split(" ")[-1]).strip()
            dados_docente_efetivo_e_auxiliar.at[index, column_label] = filtered_siape

In [88]:
dados_docente_efetivo_e_auxiliar.rename(columns={"nome_e_siape_docente": "siape_docente_efetivo", 
                                                 "nome_e_siape_docente_auxiliar": "siape_docente_auxiliar"},
                                       inplace=True)

In [89]:
dados_docente_efetivo_e_auxiliar.head()

,siape_docente_efetivo,siape_docente_auxiliar
0,2116924,NaN
1,2128584,NaN
2,2119988,NaN
3,2144097,NaN
4,1622509,NaN


In [90]:
disciplinas_ministradas.drop(columns=["nome_e_siape_docente", "nome_e_siape_docente_auxiliar"], inplace=True)

In [91]:
disciplinas_ministradas

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta
0,117006,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21504,Biologia I,27,DPAA-AL.22,Semestral
1,117007,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21507,Epidemiologia,27,DPAA-AL.25,Semestral
2,117008,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21501,História I,27,DPAA-AL.19,Semestral
3,117009,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21505,Informática Básica I,27,DPAA-AL.23,Semestral
4,117010,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21500,Língua Portuguesa I,54,DPAA-AL.18,Semestral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118794,159084,Câmpus Virtual de Ead,2016,2,20162.CTSE.APC.3,3,Curso Técnico Subsequente ao Ensino Médio em S...,Subsequente,Ensino Médio,Departamento de EaD,20562,Trabalho Escolar e Teorias Administrativas,68,DPEAD.92,Semestral
118795,159126,Câmpus Virtual de Ead,2016,2,20162.CTSE.URU.3,3,Curso Técnico Subsequente ao Ensino Médio em S...,Subsequente,Ensino Médio,Departamento de EaD,20562,Trabalho Escolar e Teorias Administrativas,68,DPEAD.92,Semestral
118796,159070,Câmpus Virtual de Ead,2016,2,20162.CTSE.AGL.3,3,Curso Técnico Subsequente ao Ensino Médio em S...,Subsequente,Ensino Médio,Departamento de EaD,20562,Trabalho Escolar e Teorias Administrativas,68,DPEAD.92,Semestral
118797,159112,Câmpus Virtual de Ead,2016,2,20162.CTSE.LUZ.3,3,Curso Técnico Subsequente ao Ensino Médio em S...,Subsequente,Ensino Médio,Departamento de EaD,20562,Trabalho Escolar e Teorias Administrativas,68,DPEAD.92,Semestral


In [92]:
disciplinas_ministradas = pd.concat([disciplinas_ministradas, dados_docente_efetivo_e_auxiliar], axis=1)

In [93]:
disciplinas_ministradas.head()

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta,siape_docente_efetivo,siape_docente_auxiliar
0,117006,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21504,Biologia I,27,DPAA-AL.22,Semestral,2116924,NaN
1,117007,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21507,Epidemiologia,27,DPAA-AL.25,Semestral,2128584,NaN
2,117008,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21501,História I,27,DPAA-AL.19,Semestral,2119988,NaN
3,117009,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21505,Informática Básica I,27,DPAA-AL.23,Semestral,2144097,NaN
4,117010,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21500,Língua Portuguesa I,54,DPAA-AL.18,Semestral,1622509,NaN


## Análise da coluna "codigo_pauta"

Segundo a pesquisa feita pelo professor Leizer, levantamos a hipótese de que a coluna "codigo_pauta" é um código único que define cada elemento de forma única em um conjunto de dados. Podemos usar uma função de distinção em para tentar comprovar essa hipótese.

In [94]:
if disciplinas_ministradas["codigo_pauta"].nunique() == disciplinas_ministradas["codigo_pauta"].count():
    print(f"Todos os valores da coluna 'codigo_pauta' são únicos.")
else:
    print(f"Os valores da coluna 'codigo_pauta' não são únicos.")

Os valores da coluna 'codigo_pauta' não são únicos.


In [95]:
codigos_pauta_duplicados = disciplinas_ministradas.duplicated(subset="codigo_pauta")
codigos_pauta_duplicados[codigos_pauta_duplicados == True]

39        True
61        True
62        True
104       True
206       True
          ... 
118793    True
118794    True
118795    True
118796    True
118797    True
Length: 9421, dtype: bool

In [96]:
disciplinas_ministradas.iloc[39, :]

codigo_pauta                                                           123122
campus_ofertante                                          Câmpus Águas Lindas
ano_letivo                                                               2014
periodo_letivo                                                              2
turma                                                       20142.CTIPE-AL.2N
periodo_disciplina                                                          2
nome_curso                  Técnico Integrado ao Ensino Médio em Enfermage...
modalidade                                                    Integrado - EJA
nivel                                                            Ensino Médio
departamento                  Departamento de Áreas Acadêmicas - Águas Lindas
codigo_disciplina                                                       21500
nome_disciplina                                           Língua Portuguesa I
carga_horaria_disciplina                                        

In [97]:
disciplinas_ministradas[disciplinas_ministradas["codigo_pauta"] == 123122]

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta,siape_docente_efetivo,siape_docente_auxiliar


Pelo que foi observado na última célula, uma possível razão para estarem ocorrendo duplicidades é a especificação de um **docente** e de um **docente auxiliar** em registros separados, ao invés de estarem em um mesmo registro.

Vamos investigar isso mais a fundo.

In [98]:
# Todas as colunas menos as duas últimas
disciplinas_duplicadas_diff = disciplinas_ministradas.duplicated(subset=[
    "codigo_pauta", 
    "campus_ofertante", 
    "ano_letivo", 
    "periodo_letivo", 
    "turma", 
    "periodo_disciplina", 
    "nome_curso", 
    "modalidade", 
    "nivel", 
    "departamento", 
    "codigo_disciplina", 
    "sigla_disciplina", 
    "nome_disciplina", 
    "carga_horaria_disciplina",
    "periodo_oferta"], keep=False)

disciplinas_duplicadas_indexes = disciplinas_duplicadas_diff[disciplinas_duplicadas_diff == True].index

disciplinas_duplicadas = disciplinas_ministradas.iloc[disciplinas_duplicadas_indexes, :].sort_values(by="codigo_pauta")

a = disciplinas_duplicadas.groupby("codigo_pauta").size()

a[a > 2]

codigo_pauta
101820    3
104767    4
105015    5
105300    6
105311    5
         ..
95751     3
95816     3
95835     3
98175     3
99491     5
Length: 854, dtype: int64

In [99]:
disciplinas_duplicadas[disciplinas_duplicadas["codigo_pauta"] == 57077]

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta,siape_docente_efetivo,siape_docente_auxiliar


Foi identificado que existem registros de disciplinas ministradas que foram ministradas por diversos professores, tanto auxiliares como efetivos. As prováveis situações para isso são:

- A disciplina trocou de professor enquanto estava sendo lecionada, por exemplo: o professor efetivo incorre de uma enfermidade grave e ganha meses de atestado, então a matéria precisa de outro professor, então um professor subsituto entra no lugar do efetivo e assim sucessivamente.

- A disciplina por alguma razão necessitou de dois docentes.

In [100]:
disciplinas_ministradas.head()

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta,siape_docente_efetivo,siape_docente_auxiliar
0,117006,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21504,Biologia I,27,DPAA-AL.22,Semestral,2116924,NaN
1,117007,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21507,Epidemiologia,27,DPAA-AL.25,Semestral,2128584,NaN
2,117008,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21501,História I,27,DPAA-AL.19,Semestral,2119988,NaN
3,117009,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21505,Informática Básica I,27,DPAA-AL.23,Semestral,2144097,NaN
4,117010,Câmpus Águas Lindas,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21500,Língua Portuguesa I,54,DPAA-AL.18,Semestral,1622509,NaN


## Substituição do câmpus pela sua sigla

In [101]:
for campus in disciplinas_ministradas["campus_ofertante"].unique():
    print(f"• {campus}")

• Câmpus Águas Lindas
• Câmpus Anápolis
• Câmpus Aparecida de Goiânia
• Câmpus Cidade de Goiás
• Câmpus Formosa
• Câmpus Goiânia
• Câmpus Goiânia Oeste
• Câmpus Inhumas
• Câmpus Itumbiara
• Câmpus Jataí
• Câmpus Luziânia
• Câmpus Senador Canedo
• Câmpus Uruaçu
• Câmpus Valparaíso
• Câmpus Virtual de Ead


In [102]:
campus_replacement_map = {
    "Câmpus Águas Lindas": "LIN",
    "Câmpus Anápolis": "ANA",
    "Câmpus Aparecida de Goiânia": "APA",
    "Câmpus Cidade de Goiás": "GOI",
    "Câmpus Formosa": "FOR",
    "Câmpus Goiânia": "GYN",
    "Câmpus Goiânia Oeste": "OES",
    "Câmpus Inhumas": "INH",
    "Câmpus Itumbiara": "ITU",
    "Câmpus Jataí": "JAT",
    "Câmpus Luziânia": "LUZ",
    "Câmpus Senador Canedo": "SEN",
    "Câmpus Uruaçu": "URU",
    "Câmpus Valparaíso": "VAL"
}

In [103]:
disciplinas_ministradas["campus_ofertante"].replace(campus_replacement_map, inplace=True)

In [104]:
for campus in disciplinas_ministradas["campus_ofertante"].unique():
    print(f"• {campus}")

• LIN
• ANA
• APA
• GOI
• FOR
• GYN
• OES
• INH
• ITU
• JAT
• LUZ
• SEN
• URU
• VAL
• Câmpus Virtual de Ead


In [105]:
disciplinas_ministradas.head()

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta,siape_docente_efetivo,siape_docente_auxiliar
0,117006,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21504,Biologia I,27,DPAA-AL.22,Semestral,2116924,NaN
1,117007,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21507,Epidemiologia,27,DPAA-AL.25,Semestral,2128584,NaN
2,117008,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21501,História I,27,DPAA-AL.19,Semestral,2119988,NaN
3,117009,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21505,Informática Básica I,27,DPAA-AL.23,Semestral,2144097,NaN
4,117010,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21500,Língua Portuguesa I,54,DPAA-AL.18,Semestral,1622509,NaN


## Cruzamento dos dados dos cursos

Alguns cursos especificados no dataset de **Disciplinas Ministradas** não estão especificados no dataset de cursos, uma provável hipótese para isso acontecer é que o curso foi extinto e parou de ser ofertado, mas nos dados de disciplinas ofertadas eles ainda constam, pois o espaço de tempo coberto por esse dataset é maior.

A decisão final foi remover os registros de **disciplinas ministradas** que se referem a cursos que não constam no dataset de cursos, objetivando uma melhor corretude dos dados. Pois caso uma disciplina que foi ministrada em um curso que não existe registros sobre ele, acaba ocorrendo a situação em que a disciplina fica com um registro órfão, pois não será possível rastrear nem o câmpus, nem o curso em que ela foi ministrada, tendo em mente o esquema abaixo para organização desses dados:

![diagrama de relacionamentos das disciplinas ministradas](./assets/images/diagrama_disciplinas_ministradas.svg)

In [106]:
cursos = pd.read_csv(
    DA_STORAGE_DIR / "preprocessed/cursos.csv",
    delimiter=";"
)

In [108]:
cursos.head()

,campus,nome,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
0,LIN,Técnico Integrado ao Ensino Médio em Análises ...,967,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3683,3186.0,120.0,160.0,108.0,3574.0,6
1,LIN,Técnico Integrado ao Ensino Médio em Meio Ambi...,968,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3684,2970.0,120.0,160.0,54.0,3304.0,6
2,LIN,Técnico Integrado ao Ensino Médio em Vigilânci...,908,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3592,3240.0,120.0,160.0,54.0,3574.0,6
3,LIN,Técnico Integrado ao Ensino Médio em Enfermage...,909,Integrado - EJA,Presencial,Noturno,Semestral,60,Ensino Médio,3593,2547.0,120.0,0.0,0.0,2667.0,8
4,LIN,Licenciatura em Ciências Biológicas,1174,Licenciatura,Presencial,Noturno,Anual,30,Superior,4316,2960.0,200.0,0.0,108.0,3268.0,8


In [109]:
disciplinas_ministradas.head()

,codigo_pauta,campus_ofertante,ano_letivo,periodo_letivo,turma,periodo_disciplina,nome_curso,modalidade,nivel,departamento,codigo_disciplina,nome_disciplina,carga_horaria_disciplina,sigla_disciplina,periodo_oferta,siape_docente_efetivo,siape_docente_auxiliar
0,117006,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21504,Biologia I,27,DPAA-AL.22,Semestral,2116924,NaN
1,117007,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21507,Epidemiologia,27,DPAA-AL.25,Semestral,2128584,NaN
2,117008,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21501,História I,27,DPAA-AL.19,Semestral,2119988,NaN
3,117009,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21505,Informática Básica I,27,DPAA-AL.23,Semestral,2144097,NaN
4,117010,LIN,2014,1,20141.CTIPE-AL.1N,1,Técnico Integrado ao Ensino Médio em Enfermage...,Integrado - EJA,Ensino Médio,Departamento de Áreas Acadêmicas - Águas Lindas,21500,Língua Portuguesa I,54,DPAA-AL.18,Semestral,1622509,NaN


In [124]:
disciplinas_ministradas_ativas = disciplinas_ministradas[disciplinas_ministradas["campus_ofertante"].isin(cursos["campus"]) 
                        & disciplinas_ministradas["nome_curso"].isin(cursos["nome"])]

disciplinas_ministradas_ativas.shape

(99813, 17)

In [122]:
disciplinas_ministradas.shape

(118799, 17)

In [125]:
disciplinas_ministradas.shape[0] - disciplinas_ministradas_ativas.shape[0]

18986